In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
from IPython.display import SVG, display

from penai.hierarchy_generation.utils import InteractivePenpotDesignVisualizer
from penai.registries.projects import SavedPenpotProject
from penai.render import WebDriverSVGRenderer
from penai.utils.vis import (
    DesignElementVisualizer,
    ShapeHighlighter,
)

plt.rcParams["figure.figsize"] = (40, 40)

# Loading and working with Penpot files

The Penai Python package provides extensive functionality to load and work with Penpot files. In this notebook, we will demonstrate how to load and inspect a penpot file, render Penpot design elements to bitmaps, derive bounding boxes for each shape and visualize the hiearchy of a board within a Penpot page.

## Exporting Penpot projects

First, the Penpot project to work with needs to be exported to the ".svg + .json"-format. In opposition to the internal binary data format, this allows us to more easily parse the project's data structures.

To export a Penpot project to this format, open a project in Penpot, then click the three dots ("...") located right of the Project title and choose `File > Download standard file (.svg + .json)`.

![](../../resources/images/penpot_project_export.png)

## Loading a Penpot project in Python

After extracting the ZIP file, the Penpot project can be loaded with the `PenpotProject.from_directory` function:

In [ ]:
# Load a project from the example project registry
project = SavedPenpotProject.MATERIAL_DESIGN_3.load(pull=True)

# OR from a project folder
# project = PenpotProject.from_directory("../../data/raw/designs/Material Design 3")

## Inspecting the Penpot project

We can inspect the project's content by simply printing the project object. Each project consists of _Penpot Files_ at the highest level, which consist of `Penpot Pages`. Each page is contains a hierarchy of _Penpot Shapes_. Other data types and structures are omitted here for simplicity but can be found in the [official Penpot documentation](https://help.penpot.app/technical-guide/developer/data-model/) on this topic.

In [ ]:
print(project)

This project contains only one file which we generally refer to as the _main file_ and can be retrieved in the following way:

In [ ]:
main_file = project.get_main_file()

The file contains several pages whose names can be accessed with the `page_names` property:

In [ ]:
main_file.page_names

## Penpot page to bitmap

An important requirement for providing Penpot design elements to AI models (i.e. Vision Language Models) is the ability to render them as bitmaps. Penpot currently uses the SVG format to display design documents in the browser which technically should make it easy to render SVGs to bitmaps.

However, due to the great complexity of the SVG standard, many open source SVG packages and particularly those for Python, implement only a relatively small part of the standard which turned to not be sufficient for us. The best solution therefore turned out to use a [headless browser](https://en.wikipedia.org/wiki/Headless_browse) for SVG rendering due to the generally good SVG support of major browsers.

The Penai project therefore implement web driver-based SVG renderer which take a SVG file or string as input and return a bitmap and possibly render artifacts such as bounding boxes.

To use this functionality, we first need to extract a Penpot page from the design file:

In [ ]:
cover_page = main_file.get_page_by_name("Cover")

In [ ]:
with WebDriverSVGRenderer.create_chrome_renderer() as renderer:
    output = renderer.render_svg(cover_page.svg)

In [ ]:
output.image

In [ ]:
with WebDriverSVGRenderer.create_chrome_renderer(infer_bounding_boxes=True) as renderer:
    output = renderer.render_svg(cover_page.svg)

In [ ]:
output.artifacts

In [ ]:
from pprint import pprint

In [ ]:
pprint(output.artifacts)

In [ ]:
shape_highlighter = ShapeHighlighter()

visualizer = DesignElementVisualizer(shape_visualizer=shape_highlighter)

In [ ]:
cover_page.svg.retrieve_and_set_view_boxes_for_shape_elements()

In [ ]:
main_frame, *_ = cover_page.svg.get_shape_elements_at_depth(0)

In [ ]:
main_frame

In [ ]:
hierarchy_visualizer = InteractivePenpotDesignVisualizer(main_frame)

In [ ]:
display(SVG(hierarchy_visualizer.svg.to_string()))